# The purpose of this notebook

In discussion with Michael, I mentioned that we support our use of a linear model of the suppression data by showing that it has better mean and median R^2 than a linear fit to the log-logged data. However, both box plots of fits showed a group of curves (each being one eye of one patient viewing the target under a certain surround/presentation condition etc) that were very badly fit by either model, like near zero and enough to be outliers on a boxplot. So, we thought maybe we'd remove them and see if that changes the results. That's what I'm attempting to do here; The immediately previous notebook, whose graphs are in redo-201901, are the comparison (these are in redo-2901902-exclude_bad_fits)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import string

import numpy as np
np.set_printoptions(precision=3)

import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import suppression as s
import utils

In [ ]:
pd.__version__

In [ ]:
sns.__version__

In [ ]:
sns

In [ ]:
gaba_fn = 'gaba_data_2019.txt'
supp_fn = 'supp_data_individual_20170427.txt'

In [ ]:
sdf = utils.load_psychophys(supp_fn)
gaba_col = 'mean_occ_all' #'motor' # or 'occ_binoc', 'mean_occ_all', 'motor'
gdf = utils.load_gaba(gaba_fn, gaba_col)

In [ ]:
gdf.Population = gdf.Population.astype('category')
gdf.Population.cat.categories # 0 AMB 1 CON
gdf.Population = gdf.Population.cat.rename_categories(['Persons with Amblyopia', 'Normally-sighted individuals'])

### Set variables used for graphing

In [ ]:
colors_amb = ["#3274a1","#72b4e1"]
colors_con = ["#e1812c", "#ffc68c"]
colors4 = colors_amb + colors_con
traces4 = ['Amblyope-De', 'Amblyope-Nde', 'Control-De', 'Control-Nde']
traces_graph4 = [f"Persons with\nAmblyopia, DE", f"Persons with\nAmblyopia, NDE", \
                 f"Normally-sighted\nindividuals, DE", f"Normally-sighted\nindividuals, NDE"]
plot_dir = f"plots/redo-201910-{gaba_col}-bootstrap"

## Analyze tasks separately (before subsetting to include common subjects)##

In [ ]:
pp_subjs = np.unique(sdf.Subject)
n_pp_subjs = len(pp_subjs)
gaba_subjs = np.unique(gdf.subjName)
n_gaba_subjs = len(gaba_subjs)
print(f"Psychophysics subjects (n={n_pp_subjs}):\n", pp_subjs)
print(f"GABA subjects (n={n_gaba_subjs}):\n", gaba_subjs)

# GABA only analyses

### GABA t-test, CON v AMB

In [ ]:
gdf

In [ ]:
pop_group = gdf.groupby("Population")
pop_group.describe(percentiles=[.5])

In [ ]:
gaba_per_group = [col for col_name, col in pop_group['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group, nan_policy='omit')
print(tstat, pval)

**Thus we find no significant difference in GABA levels between CON and AMB.**

### GABA violin plot, all subjects

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_diffs_n{n_gaba_subjs}_{gaba_col}.pdf") as pdf:
    fig = plt.figure(figsize=(8,8))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.violinplot(y='GABA',x='Presentation',hue='Population',data=gdf,split=True,inner='stick',ax=ax,legend=False)
    #ax.legend_.remove()
    ax.xaxis.set_visible(False)
    ax.set_ylabel('GABA:Creatine ratio')
    #ax.set_yticklabels([])
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')

### Note: the subject with the lowest GABA:Cr ('tt', .162) is not in the psychophysics data

# Select one psychophysical task's data #

In [ ]:
task = 'SS'
sdf = sdf[sdf['Task']==task]

df_to_model = sdf.copy() # make a deep copy

In [ ]:
n_pp_subjs_thistask = len(np.unique(df_to_model.Subject))
amb_subjs = np.unique(df_to_model[df_to_model["Population"]=="Amblyope"]["Subject"])
print(amb_subjs)
n_amb_subjs_thistask = len(amb_subjs)
print(f"There are {n_pp_subjs_thistask} subjects for Task {task}, of which {n_amb_subjs_thistask} are Amblyopes.")

# Modeling

In [ ]:
df_to_model.head()

### Begin grouping data into conditions to model Subject's ThreshElev as a function of logRelContrast #

In [ ]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace'] # One condition
pp_gvars_base = pp_gvars + ['BaselineThresh']

groups_with_baseline = df_to_model.groupby(pp_gvars_base)

# Check if there are any conditions with only two data points
for gv, gr in groups_with_baseline:
    if len(gr)<=2:
        print(gv, gr)

#### BaselineThresh analysis before we exclude bad fits; since this is observed not modeled its ok

In [ ]:
print(pp_gvars_base)
pp_gvars_base_agg = [v for v in pp_gvars if v != 'Subject']

In [ ]:
pp_gvars_base_agg

In [ ]:
for gv, g in df_to_model.groupby(pp_gvars_base_agg):
    print(gv, len(np.unique(g['BaselineThresh'])))

In [ ]:
df_to_model.groupby(['Task', 'Orientation', 'Presentation', 'Population']).apply(utils.test_baseline_diffs)

In [ ]:
baseline_df = df_to_model.groupby(pp_gvars_base_agg).apply(utils.describe_baselines).reset_index()

In [ ]:
baseline_df_reduced = baseline_df[(baseline_df.Orientation=='Cross') & (baseline_df.Presentation=='nMono')]
baseline_df_reduced

In [ ]:
baseline_plot_df = utils.make_baseline_df_to_plot(baseline_df_reduced)

In [ ]:
baseline_plot_df.head()

In [ ]:
with s.PdfPages(f"{plot_dir}/{task}_baseline_diffs.pdf") as pdf:
    fig = plt.figure(figsize=(8,8))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    x_pos = np.arange(len(baseline_plot_df['Trace']))
    plt.bar(x_pos, baseline_plot_df['mean'], data=baseline_plot_df, yerr='SEM', color=colors4)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(traces_graph4)
    ax.set_ylabel('Baseline Contrast Threshold (C%)')
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')
    

### Linear model using statsmodels

In [ ]:
lin_results = groups_with_baseline.apply(utils.linear_fit_params, 'RelMaskContrast', 'ThreshElev').reset_index()

In [ ]:
lin_results.head(n=16)

In [ ]:
sns.distplot(lin_results.rsquared, kde=False, rug=True)

In [ ]:
np.count_nonzero(lin_results.rsquared>.999999)

In [ ]:
lin_results.rsquared.mean()

In [ ]:
lin_results.boxplot(column='rsquared', by=['Orientation', 'Presentation'], grid=False, figsize=(16, 4))

In [ ]:
lin_results.boxplot(column='rsquared', by=['Orientation', 'Population', 'Presentation'], grid=False, figsize=(16, 4))

### Fit the log-logged data to see if that's better

In [ ]:
log_results = groups_with_baseline.apply(utils.linear_fit_params, 'logRelMaskContrast', 'logThreshElev').reset_index()

log_results.rsquared.mean()

In [ ]:
log_results.boxplot(column='rsquared', by=['Orientation', 'Population', 'Presentation'], grid=False, figsize=(16, 4))

## Conclusion: linear fits are overall better.

## Identify subjects with negative slope (for Jian)

In [ ]:
neg_slopes = lin_results[lin_results['slope']<0]
neg_slopes

In [ ]:
neg_slopes.Subject.unique()

## Exclude bad fits (new 2019-02-25)

In [ ]:
len(lin_results)

In [ ]:
lin_results_exc = lin_results.groupby(['Task', 'Population']).apply(utils.remove_outliers_halfvar).reset_index(drop=True)

In [ ]:
lin_results_exc.rsquared.min()

In [ ]:
lin_results_exc.Subject.value_counts().sum()

In [ ]:
231/256

In [ ]:
pp_subs_exc = lin_results_exc.Subject.unique()
print(f'There are {len(pp_subs_exc)} unique subjects who have at least one condition of data.')

## Actually use the linear model to predict thresholds

In [ ]:
lin_preds = groups_with_baseline.apply(utils.linear_fit_predictions, 'RelMaskContrast', 'ThreshElev').reset_index()

In [ ]:
lin_preds.columns

In [ ]:
lin_preds.head()

In [ ]:
plot_df = pd.merge(df_to_model, lin_preds, on=pp_gvars_base + ['RelMaskContrast'])

plot_df.head()

### Plot observed values and model fits

In [ ]:
#s.group_facet_plots(plot_df, s.subject_fit_plot,
#                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TOP.pdf",
#                    ['Task','Orientation','Presentation'], #each combo of this gets its own page
#                    row='Population',col='Eye',# facet rows and columns
#                    x="RelMaskContrast", y="ThreshElev", # x, y
#                    hue="Subject",yerr='ThreshElev_SE',fmt_obs='.',fmt_pred='x:',Ycol="ThreshPred") 

In [ ]:
#s.group_facet_plots(plot_df, s.population_fit_plot,
#                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TO.pdf",
#                    ['Task','Orientation'], #each combo of this gets its own page
#                    row='Presentation',col='Eye',# facet rows and columns
#                    x="RelMaskContrast", y="ThreshElev", # x, y
#                    hue="Population",yerr='ThreshElev_SE',fmt_obs='.',fmt_pred='x:',Ycol="ThreshPred") 

## Now exclude the predictions for the bad fits

In [ ]:
lin_results.Orientation.value_counts()

In [ ]:
lin_results_exc.Orientation.value_counts()

In [ ]:
lin_results_exc.Population.value_counts()

In [ ]:
lin_results.Population.value_counts()

In [ ]:
comb_rsq_preds = pd.merge(lin_results_exc, lin_preds, on=pp_gvars_base)

In [ ]:
comb_rsq_preds.rsquared.min()

In [ ]:
comb_rsq_preds.head()

### Pick an xvalue (RelMaskContrast) to evaluate models at

 * 2018-09-24: Abandoning Eunice's binning. Instead, try to figure out a good RelMaskContrast programatically.
   * Just looking at it via describe(), i'd say somewhere between 5 and 10 -- probably 6 (for SS) and 10 (for OS)
 * 2018-10-08: My previous approach was too subjective. Instead, evaluate model at various percentiles...
   * do this separately for Task, Orientation, Presentation (so pick 8 total numbers)
   * At this point it's easier to just use the statsmodels.ols functions maybe? The way it's currently done is a legacy that allows different models to be swapped in... which I hope to god is not going to be the direction we go in again.
   * nvm, used the lmfit solution since it returned a nicely formatted pfit df and is tested to work
   * Here I want to take the slope and y-int and calculate the model prediction at the specified percentiles above (0-1, increments of 0.2). So, first calculate the RelMC at each of those percentiles, then apply it like below.


In [ ]:
percentile_bins = np.linspace(0, 1, num=11)

In [ ]:
percentile_bins

#### Get the interpolated RelMaskContrasts for each regression line, i.e. the range of x-values

In [ ]:
relmc_pcts_df = comb_rsq_preds.groupby(pp_gvars_base)['RelMaskContrast'].describe(percentiles=percentile_bins)

In [ ]:
# fix stupid column naming from describe()
relmc_pcts_df.columns = [f"{int(float(col[:-1])):03d}" if col[-1]=="%" else col for col in relmc_pcts_df.columns]

In [ ]:
relmc_pcts_df.columns

In [ ]:
relmc_pcts_df = relmc_pcts_df.filter(regex='0|1')
relmc_pcts_df.head()

In [ ]:
relmc_pcts_df.columns

In [ ]:
relmc_pcts_df_melted = relmc_pcts_df.reset_index().melt(id_vars=pp_gvars_base, var_name='percentile', value_name='RelMaskContrast_pct')

In [ ]:
relmc_pcts_df_melted.head()

In [ ]:
predict_pcts_df = pd.merge(lin_results_exc, relmc_pcts_df_melted, on=pp_gvars_base)

In [ ]:
predict_pcts_df.head()

In [ ]:
predict_pcts_df['percentile'] = predict_pcts_df['percentile'].astype(int)
predict_pcts_df['RelMaskContrast_pct'] = predict_pcts_df['RelMaskContrast_pct'].astype(float)
predict_pcts_df['relmc_bin'] = (np.around(predict_pcts_df['RelMaskContrast_pct'])).astype(int)
predict_pcts_df['ThreshElev_pct'] = (predict_pcts_df['y_int'] + (predict_pcts_df['RelMaskContrast_pct']*predict_pcts_df['slope'])).astype('float')

In [ ]:
predict_pcts_df.head()

In [ ]:
predict_pcts_df.rsquared.min()

In [ ]:
gvars_test = ['Task','Orientation','Presentation','Population']
# equal_var=False makes it Welch's t-test, which does not assume the groups have equal variance
selected_bin_df = utils.find_pct_to_predict(predict_pcts_df, gvars_test,
                    'relmc_bin', 'ThreshElev_pct', test_func=st.ttest_ind, equal_var=False)

In [ ]:
g_TOP = selected_bin_df.groupby(['Task', 'Orientation', 'Presentation'])
def set_relmctopred_to_amb_val(g):
    ambs = g[g['Population']=='Amblyope']
    assert(np.all(ambs['RelMCToPred']==ambs['RelMCToPred'].iat[0]))
    assert(np.all(ambs['BinNumberToPred']==ambs['BinNumberToPred'].iat[0]))
    g['RelMCToPred'] = ambs['RelMCToPred'].iat[0]
    g['BinNumberToPred'] = ambs['BinNumberToPred'].iat[0]
    return g
selected_bin_df = g_TOP.apply(set_relmctopred_to_amb_val).reset_index()

In [ ]:
selected_bin_df.groupby(gvars_test)['RelMCToPred','ThreshElev_pct'].describe()

In [ ]:
selected_bin_df['ThreshPredCritical'] = selected_bin_df['y_int'] + selected_bin_df['slope'] * selected_bin_df['RelMCToPred']
selected_bin_df['ThreshPredCriticalUnnorm'] = selected_bin_df['ThreshPredCritical'] * selected_bin_df['BaselineThresh']

In [ ]:
selected_bin_df

### Melt the result of the modeling into long format for plotting

In [ ]:
pfit_all_ppsub = pd.melt(selected_bin_df, id_vars=[*pp_gvars, 'rsquared'],
                    value_vars=['BaselineThresh', 'y_int', 'slope', 'ThreshPredCritical', 'ThreshPredCriticalUnnorm'],
                    var_name='measure')
pfit_all_ppsub.head()

In [ ]:
pp_stats = pfit_all_ppsub[(pfit_all_ppsub.measure=="ThreshPredCritical") | 
                          (pfit_all_ppsub.measure=="slope")].drop_duplicates()

In [ ]:
pp_slopes = pfit_all_ppsub[pfit_all_ppsub.measure=="slope"].drop_duplicates()

In [ ]:
pp_stats.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'Eye', 'Trace','measure'])['value'].describe(percentiles=[.5])

In [ ]:
def test_suppression_diffs(g):
    ndes = np.unique(g[g.Eye=='Nde']['value'])
    des = np.unique(g[g.Eye=='De']['value'])
    #g.hist()
    print(len(ndes), ' ', len(des))
    print(st.ttest_ind(ndes, des))
    return st.ttest_ind(ndes, des)

gs = pp_stats.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'measure'])
for gv, g in gs:
    print(gv)
    test_suppression_diffs(g)

## Subset to include only (GABA and psychophyics) subjects

In [ ]:
gaba_and_pp_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
n_gaba_and_pp_subjs = len(gaba_and_pp_subjs)

In [ ]:
sdf = sdf[sdf.Subject.isin(gaba_and_pp_subjs)] # only subjects who did _the current_ pp task and GABA
gaba_and_pp_subjs_thistask = np.unique(sdf.Subject)
n_gaba_and_pp_subjs_thistask = len(gaba_and_pp_subjs_thistask)
print(f"Of the {n_gaba_and_pp_subjs} subjects with both GABA and psychophysics data, {n_gaba_and_pp_subjs_thistask} have both for task {task}.\n{gaba_and_pp_subjs_thistask}")

### Remove subjects we don't have data on both GABA/PP for

In [ ]:
gdf = gdf[gdf.subjName.isin(sdf.Subject)] # only subjects who did both tasks
amb_subjs = (gdf[gdf.Population=='Persons with Amblyopia'])
print(f'Of the {len(gdf)} subjects with GABA and {task} data, {len(amb_subjs)} are Amblyopes.')
n_this_task = len(gdf)

In [ ]:
stats_thistask = lin_results_exc[lin_results_exc.Subject.isin(gaba_and_pp_subjs_thistask)].groupby(['Subject'])['rsquared'].describe()

In [ ]:
stats_thistask

In [ ]:
n_gaba_and_pp_subjs_thistask * 8 - stats_thistask['count'].sum()

## Combine Psychophysics and GABA below

In [ ]:
#Grab the GABA measure for each subject and append it to each observation for easy plotting
comb = pfit_all_ppsub.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])
comb.drop_duplicates(inplace=True)

#subset to include only those subjects with GABA data
comb_gabappsub = comb[~np.isnan(comb['GABA'])]
print(len(comb), len(comb_gabappsub))


In [ ]:
comb_gabappsub.head(n=10)

### Add copies of rows to simulate weighting by rsquared

 * (bypassed by modifying seaborn's bootstrap, 2019-08-22)
 * (weighting dropped 2019-08-26 after MAS meeting)

In [ ]:
#rep = np.ones(len(comb_gabappsub))
#times = np.around(comb_gabappsub['rsquared']*10)
#.iloc[np.arange(len(comb_gabappsub))]
#print(rep, times, rep*times)
#comb_gabappsub_weighted = comb_gabappsub.iloc[np.arange(len(comb_gabappsub)).repeat(times)]

### Do Spearman's R

In [ ]:
spearman_df = comb_gabappsub[(comb_gabappsub.measure == 'BaselineThresh') | 
                             (comb_gabappsub.measure == 'ThreshPredCritical')]

In [ ]:
plot_groups = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure','Eye','Trace'])
for gv, gr in plot_groups:
    #if 'nDicho' in gv:
    print(gv)
    print(stats.spearmanr(gr.GABA, gr.value))

In [ ]:
#graphs!
colors_a = ["#3274a1","#72b4e1"]
colors_c = ["#e1812c", "#ffc68c"]
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}.pdf") as pdf:
    plot_groups = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure'])
    plot_groups_eacheye = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure','Eye'])
    for gv, gr in plot_groups:
        #if gv[2] == 'nDicho':
        print(gv, np.all(np.isnan(gr['value'])), len(gr['value']))
        pal = [colors_a if 'Amblyope' in gv else colors_c]
        g2 = s.gaba_vs_psychophys_plot_2line_nofacet(gv, gr, palette=pal[0], aspect=1.2)
        pdf.savefig(g2.fig)

plt.close('all')

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}_ind.pdf") as pdf:
    for gv, gr in plot_groups:
        print(gv)
        g = s.gaba_vs_psychophys_plot(gv, gr, hue="Eye",
                    palette=dict(De="g", Nde="m"),
                    n_boot=1000, height=8, aspect=1, legend_out=True, truncate=True)
        g.fig.suptitle(f"{gv} unweighted", fontsize=10, y=0.999)
        pdf.savefig(g.fig)
        plt.close('all')

In [ ]:
facet_groups = spearman_df.groupby(['Task','Orientation','Presentation','measure'])
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}_facet.pdf") as pdf:
    for gv, gr in facet_groups:
        print(gv)
        g = s.gaba_vs_psychophys_plot(gv, gr, 
                    legend_box=[0.89, 0.60, 0.1, 0.1],
                    col="Population", hue="Eye",
                    palette=dict(De="g", Nde="m"),
                    n_boot=1000, truncate=True)#, legend=False)
        g.fig.suptitle(f"{gv}", fontsize=10, y=0.999)
        pdf.savefig(g.fig)
        plt.close('all')

In [ ]:
rdiffs_nores = facet_groups.apply(utils.compare_rs, n_boot=1000, resample=False).reset_index()\
            .rename(columns={"level_4":"iteration"})

In [ ]:
rdiffs = rdiffs_nores
rdiffs.head()

In [ ]:
rdiffs_supp = rdiffs[rdiffs['measure']=="ThreshPredCritical"]

In [ ]:
rdiffs_supp.boxplot(column='amb_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
rdiffs_supp.hist(column='amb_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
rdiffs_supp.boxplot(column='con_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
rdiffs_supp.boxplot(column='pop_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
comb_gabappsub.head()

In [ ]:
spearman_df.measure.unique()

In [ ]:
kelly_file = f"{plot_dir}/{task}_data_frame.csv"
spearman_df.to_csv(kelly_file)

### Orientation Selective Suppression

In [ ]:
oss_gvars = ["Task", "Presentation", "Population", "Subject", "Eye", "Trace",
             "measure", "GABA"]
oss_gvars_combeyes = ["Task", "Presentation", "Population", "Subject", 
             "measure", "GABA"]

In [ ]:
for gv, g in spearman_df.groupby(oss_gvars):
    print(gv, g.Orientation.unique())

In [ ]:
oss_df = spearman_df[spearman_df.measure=='ThreshPredCritical'].groupby(oss_gvars).apply(utils.calculate_orientation_selective_suppression).reset_index()

In [ ]:
print(np.count_nonzero(np.isnan(oss_df.value)), len(oss_df.value))

In [ ]:
oss_df.head(n=15)

In [ ]:
def oss_mean_combeyes(df, **kwargs):
    if len(df.Eye.unique())==2:
        v1 = df[df.Eye=='Nde']['value'].iloc[0]
        v2 = df[df.Eye=='De']['value'].iloc[0]
        oss_mean_combeyes = np.mean([v1, v2])
    else:
        oss_mean_combeyes = np.nan
    print(f"OSS mean across eyes: {oss_mean_combeyes}")
    return pd.Series(oss_mean_combeyes, ['value'])

In [ ]:
for gv, g in oss_df.groupby(oss_gvars_combeyes):
    print(gv, g, sep="\n")

In [ ]:
oss_df_combeyes = oss_df.groupby(oss_gvars_combeyes).apply(oss_mean_combeyes).reset_index()

In [ ]:
oss_df_combeyes

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_oss.pdf") as pdf:
    temp_df = oss_df.copy()
    temp_df['Eye'] = temp_df['Eye'].astype('category')
    plot_groups = temp_df.groupby(['Task', 'Presentation', 'Population','measure'])
    for gv, gr in plot_groups:
        #if "BaselineThresh" in gv: continue
        if "Amblyope" in gv:
            pal = colors_a
        elif "Control" in gv:
            pal = colors_c
        else:
            print('Error! neither amb nor con!')
        print(gv, np.all(np.isnan(gr['value'])))
        g2 = s.gaba_vs_psychophys_plot_2line_nofacet(gv, gr, palette=pal)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_oss_combeyes.pdf") as pdf:
    temp_df = oss_df_combeyes.copy()
    plot_groups = temp_df.groupby(['Task', 'Presentation', 'Population','measure'])
    for gv, gr in plot_groups:
        #if "BaselineThresh" in gv: continue
        if "Amblyope" in gv:
            pal = colors_a
            print(gv, np.all(np.isnan(gr['value'])))
        elif "Control" in gv:
            pal = colors_c
            print(gv, np.all(np.isnan(gr['value'])),
                  stats.spearmanr(gr.GABA, gr.value, nan_policy='omit'), sep='\n')
        else:
            print('Error! neither amb nor con!')
        g2 = s.oss_plot_2eye(gv, gr, palette=pal)
        pdf.savefig(g2.fig)
        
    plt.close('all')

### Combine measures across the two eyes

 * Does it make sense to combine all measures across both eyes (i.e. by subtracting?) For example, ThreshElev is in units of baseline, and the baseline varies by eye. So perhaps only a few measures should be combined -- say, slope/yint, ThreshPredCriticalUnnorm. 

In [ ]:
measures = comb_gabappsub[comb_gabappsub["measure"].isin(["BaselineThresh","ThreshPredCritical"])]

In [ ]:
np.unique(measures.measure)

In [ ]:
paired_obs = measures.groupby(['Task', 'Orientation', 'Population', 'Presentation', 'Subject', 'measure'])

def get_eyediff_value(g):
    if len(g)==2: # this will exclude paired observations where there was no data for one eye
        value_diff = g[g['Eye']=='Nde'].value.iat[0] - g[g['Eye']=='De'].value.iat[0]
        #print(g.name, value_diff)
        return pd.Series([value_diff], ['Nde-De'])
    else:
        print(f"Skipping because one eye is missing...")

In [ ]:
obs_diff = paired_obs.apply(get_eyediff_value).reset_index()

In [ ]:
obs_diff[obs_diff.Subject=='em']

In [ ]:
comb_botheyes = obs_diff.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
comb_botheyes

In [ ]:
print(len(np.unique(comb_botheyes.Subject)))

In [ ]:
test_groups = comb_botheyes.groupby(['Task','Orientation','Presentation','Population','measure'])
for gv, gr in test_groups:
    #print(gr.head())
    if gv[-1]=="ThreshPredCritical":
        print(gv)
        print(stats.spearmanr(gr.GABA, gr['Nde-De']))

In [ ]:
#graphs!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv)
        g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
# more graphs for presentation!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}_poster.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','Presentation','measure'])
    for gv, gr in plot_groups:
        if gv[-2] in ["nDicho"]: # use this line to exclude measures we don't want
            print(gv)
            g2 = s.gaba_vs_psychophys_plot_2line_2eye_nofacet(gv, gr, hue="Population", height=5, aspect=1.2, legend=False)
            #print(g2.axes)
            pdf.savefig(g2.fig)
        
    plt.close('all')